In [58]:
using CSV, DataFrames, Statistics, StatsBase, Distributions, JuMP, Gurobi, LinearAlgebra

const GRB_ENV = Gurobi.Env()

Academic license - for non-commercial use only - expires 2022-09-11


Gurobi.Env(Ptr{Nothing} @0x00007f8eefb97200, false, 0)

In [190]:
##### SAFI #####

X_test_adaptive = CSV.read("data/X_test_adaptive.csv", DataFrame)
y_test = CSV.read("data/y_test_speed.csv", DataFrame)
# select!(X_test_adaptive, Not([:RANSACRegressor, :GaussianProcessRegressor, :KernelRidge, :Lars, :AdaBoostRegressor,
#                      :DummyRegressor, :ExtraTreeRegressor, :Lasso, :LassoLars, :PassiveAggressiveRegressor]))

X = X_test_adaptive#[!,1]
#X = X[!,[2, 3,5,8,9,11,12,13,25]]
y = y_test[!, "speed"];

In [194]:
n = size(y)[1]
y_safi = (y .- mean(y[1:floor(Int, n/2),:], dims =1))./ std(y[1:floor(Int, n/2),:], dims=1);

In [11]:
describe(X_test_adaptive[:,2:7] .- y)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Float64,Float64,Float64,Int64,DataType
1,NUMTECH_speed,0.0329076,-4.63357,0.0733019,5.13105,0,Float64
2,speed,0.0566938,-5.07005,0.104789,3.51963,0,Float64
3,DT_speed,0.0672517,-5.52488,0.106152,5.08333,0,Float64
4,TABNET_speed,0.0081165,-4.80957,0.0602244,3.33884,0,Float64
5,RIDGE_speed,0.0248569,-5.41145,0.114219,3.64899,0,Float64
6,LASSO_speed,0.0176461,-4.4157,0.117052,3.32267,0,Float64


In [13]:
describe(X_test_adaptive[:,2:7] .- y, :mean, :std) 

,variable,mean,std
,Symbol,Float64,Float64
1,NUMTECH_speed,0.0329076,1.26841
2,speed,0.0566938,0.877551
3,DT_speed,0.0672517,1.02984
4,TABNET_speed,0.0081165,0.905541
5,RIDGE_speed,0.0248569,0.999924
6,LASSO_speed,0.0176461,0.978456


In [25]:
##### ENERGY #####
X_energy = CSV.read("data/energy_predictions_test_val.csv", DataFrame)
y_energy = CSV.read("data/energy_y_test_val.csv", DataFrame, header = 0);
describe(X_energy[4000:8000,2:25] .- y_energy[4000:8000,1], :mean, :std) 

,variable,mean,std
,Symbol,Float64,Float64
1,BayesianRidge,3.01734,59.6378
2,ElasticNet,1.44953,64.0703
3,ElasticNetCV,0.931037,60.2554
4,GammaRegressor,0.057711,68.6374
5,GeneralizedLinearRegressor,0.582401,67.0447
6,HistGradientBoostingRegressor,20.8497,65.9726
7,HuberRegressor,-5.2734,60.1917
8,LarsCV,1.13659,59.9771
9,Lasso,1.4169,59.9211


In [33]:
describe(abs.(X_energy[4000:8000,2:25] .- y_energy[4000:8000,1]), :mean, :std) 

,variable,mean,std
,Symbol,Float64,Float64
1,BayesianRidge,28.553,52.4433
2,ElasticNet,30.3495,56.4427
3,ElasticNetCV,27.5846,53.5768
4,GammaRegressor,32.5871,60.4062
5,GeneralizedLinearRegressor,32.3298,58.7354
6,HistGradientBoostingRegressor,41.7822,55.1454
7,HuberRegressor,24.0737,55.4182
8,LarsCV,26.6521,53.7404
9,Lasso,26.787,53.6174


In [196]:
size(y_safi)

(8494, 1)

In [198]:
4247-900

3347

In [488]:
N0 = 900
Nt = 100
p = 10

δ = 0.5
δ_pert = 0.5
σ_pert = 1
#y = randn(N0+Nt)
n0 = floor(Int, size(y_safi)[1]/2)
y_true = y_safi[n0-N0:n0+Nt-1]
y = copy(y_true)
d = Uniform(-δ, δ)
biases = rand(d, p)
variances = rand(p)
gradual_after = false
gradual_before = true

#Create the features X by taking y and adding noise.
X = zeros(N0+Nt, p)

for i=1:p
    d = Normal(biases[i], variances[i])
    X[:,i] = y.+ rand(d, N0+Nt)
end

#### Perturbations on base learners because of data drift
d = Normal(0, δ_pert)
biases_perturb = rand(d, p)
d = Uniform(0, σ_pert)
variances_perturb = rand(d, p)

if gradual_after
    for i=1:p
        d = Normal(biases_perturb[i], variances_perturb[i])
        #We add the perturbation for each model 
        #We make the perturbations more and more intense across time
        X[N0+1:N0+Nt,i] = X[N0+1:N0+Nt,i].+rand(d, Nt).*[t/Nt for t=1:Nt]
    end
end

if gradual_before
    for i=1:p
        for t=1:Nt
            #We compute the perturbation for each model 
            #We make the perturbations more and more intense across time
            d = Normal(t/Nt*δ_pert, t/Nt*σ_pert)
            X[N0+t,i] = X[N0+t,i]+rand(d)
        end
    end
else
    for i=1:p
        d = Normal(biases_perturb[i], variances_perturb[i])
        #We add the perturbation for each model 
        #We make the perturbations more and more intense across time
        X[N0+1:N0+Nt,i] = X[N0+1:N0+Nt,i].+rand(d, Nt)
    end
end

In [250]:
#### Perturbations on y
perturb_y = true
perturb_y_norm = true
y_bias = -0.1
y_var = 0.5

if perturb_y
    if perturb_y_norm
        d = Normal(y_bias, y_var)
        y[N0+1:N0+Nt] = y[N0+1:N0+Nt] .+ rand(d, Nt)
    else
        d = Uniform(-y_bias, y_bias)
        y[N0+1:N0+Nt] = y[N0+1:N0+Nt] .+ rand(d, Nt)
    end
end
;

In [490]:
split_ = N0/(N0+Nt)
past = 10
num_past = floor(Int, Nt/past)

last_yT = false
max_cuts = 10
verbose = false

uncertainty = 0.#1
δ_inf = 0.02
ϵ_inf = 0.02

ϵ_l2 = 0.05
δ_l2 = 0.05

reg = 1/(past*num_past)
ρ = 0.1
val = Nt-1; #n-split_index;

fix_β0 = false
more_data_for_β0 = false
benders = false
ridge = true

eval_method(X, y, y_true, split_, past, num_past, val, uncertainty, ϵ_inf, δ_inf, last_yT, 
        ϵ_l2, δ_l2, ρ, reg, max_cuts, verbose, 
        fix_β0, more_data_for_β0, benders, ridge, linear_adapt)

(3, 12)
### β0 Baseline ###
MAE Baseline: 0.2926726628350745
CVAR 0.05 :0.9516292171887668
CVAR 0.15 :0.7810909687051267
R2 : 0.5596709463338037

### β0 Baseline Retrained ###
MAE Baseline: 0.2366397874232855
CVAR 0.05 :0.7800146141813811
CVAR 0.15 :0.5965179535644068
R2 : 0.7228504073292392

### β0 V0 Linear rule Adaptive ###
MAE Baseline: 0.2906279546385435
CVAR 0.05 :0.9926259187816489
CVAR 0.15 :0.7719802373504452
R2 : 0.5620597548454742

### β0 V0 Linear rule Adaptive Retrained ###
MAE Baseline: 0.25762201468040113
CVAR 0.05 :0.8801040054421982
CVAR 0.15 :0.6322623660372929
R2 : 0.6717637589787904

### β0 Adaptive ###
MAE 0: 0.24097862855768679
CVAR 0.05 :0.8013637405312338
CVAR 0.15 :0.6107513493064963
R2 : 0.7087045512821339

### βt Adaptive ###
MAE t: 0.2068937242192836
CVAR 0.05 :0.7503999427946417
CVAR 0.15 :0.5723872188663088
R2 : 0.7651533718100071


In [251]:
split_ = N0/(N0+Nt)
past = 20
num_past = floor(Int, Nt/past)

last_yT = false
max_cuts = 10
verbose = false

uncertainty = 0.#1
δ_inf = 0.02
ϵ_inf = 0.02

ϵ_l2 = 0.05
δ_l2 = 0.05

reg = 1/(past*num_past)
ρ = 0.1
val = Nt-1; #n-split_index;

fix_β0 = false
more_data_for_β0 = false
benders = false
ridge = true

eval_method(X, y, y_true, split_, past, num_past, val, uncertainty, ϵ_inf, δ_inf, last_yT, 
        ϵ_l2, δ_l2, ρ, reg, max_cuts, verbose, 
        fix_β0, more_data_for_β0, benders, ridge)


### β0 Baseline ###
MAE Baseline: 0.23292285353920797
CVAR 0.05 :0.8248781648196017
CVAR 0.15 :0.6146000206902295
R2 : 0.7231483653113947

### β0 Baseline Retrained ###
MAE Baseline: 0.24235092030157646
CVAR 0.05 :0.8089715074444982
CVAR 0.15 :0.6809646681941373
R2 : 0.686903282141481

### β0 Adaptive ###
MAE 0: 0.20902448857494324
CVAR 0.05 :0.7269394354418665
CVAR 0.15 :0.5684294172269506
R2 : 0.7696332848022117

### βt Adaptive ###
MAE t: 0.21357895684413064
CVAR 0.05 :0.7495916668128307
CVAR 0.15 :0.5651026247906372
R2 : 0.76674460974741


In [59]:
function get_Y(X, t)
    "
    Create the vector of data for the dual problem
    "
    T, p = size(X)
    Y = zeros(1,T*p)
    Y[(t-1)*p+1:t*p] = X[t,:]
    return Y
end

function get_Z(X)
    T, p = size(X)
    Z = zeros(T, T*p)
    for t=1:T
        Z[t,:] = get_Y(X,t)
    end
    return Z
end

function get_A(X, t)
    T, p = size(X)
    A = zeros(p,T*p)
    A[:,(t-1)*p+1:t*p] = 1 * Matrix(I, p, p)
    return A
end


function solve_model_benders(m)
    "Solve the Benders master problem
    "
    optimize!(m)
    U_OA = objective_value(m)
    #println("U_OA from solve ", U_OA)
    return value.(m[:β]), value.(m[:α]), U_OA
end

function S_primal(X, y, β0, epsilon, delta)

    n, p = size(X)

    # Create model
    model = Model(with_optimizer(Gurobi.Optimizer, GRB_ENV))
    set_optimizer_attribute(model, "OutputFlag", 0)

    # Add variables
    @variable(model, β[i=1:n,j=1:p])
    @variable(model, b[i=1:n]>=0)

    # Add objective
    @objective(model, Min, sum(b[i] for i=1:n))

    #@constraint(model,[i=1:n], y .- dot(X,β) .<= b)
    #@constraint(model,[i=1:n],-y .+ dot(X,β) .<= b)


    @constraint(model, res_plus[i=1:n],  - y[i] + dot(X[i,:],β[i,:]) <= b[i])
    @constraint(model, res_minus[i=1:n],  y[i] - dot(X[i,:],β[i,:]) <= b[i])

    @constraint(model, diff_plus[i=2:n],   β[i,:] .- β[i-1,:] .<= delta)
    @constraint(model, diff_minus[i=2:n], - β[i,:] .+ β[i-1,:] .<= delta)

    @constraint(model, diff_0_plus[i=1:n],   β[i,:] .- β0 .<= epsilon)
    @constraint(model, diff_0_minus[i=1:n], - β[i,:] .+ β0 .<= epsilon)

    optimize!(model);

    return objective_value(model), getvalue.(β)
end


function R(X, D_min, D_max, β0, epsilon, delta)
    "
    Full dual problem
    "
    T, p = size(X)
    Z = get_Z(X)

    # Create model
    model = Model(with_optimizer(Gurobi.Optimizer, GRB_ENV))#Model(with_optimizer(Gurobi.Optimizer))
    set_optimizer_attribute(model, "OutputFlag", 0)
    set_optimizer_attribute(model, "NonConvex", 2)

    # Add variables
    @variable(model, λ[i=1:2, j=1:T] >= 0)
    @variable(model, ν[i=1:2, j=1:T-1, k=1:p]>=0)
    @variable(model, μ[i=1:2, j=1:T, k=1:p]>=0)

    @variable(model, y[j=1:T])


    @constraint(model,[t=1:T], λ[1,:] .+ λ[2,:] .== 1)


    @constraint(model, transpose(λ[2,:])*Z-transpose(λ[1,:])*Z
                        + sum(transpose(ν[1,t,:])*(get_A(X, t+1).-get_A(X, t)) for t=1:T-1)
                        + sum(transpose(ν[2,t,:])*(-get_A(X, t+1).+get_A(X, t)) for t=1:T-1)
                        + sum(transpose(μ[1,t,:])*get_A(X,t) for t=1:T)
                        - sum(transpose(μ[2,t,:])*get_A(X,t) for t=1:T) .== 0)

    #y in uncertainty set
    @constraint(model, [1:T], D_min .<= y)
    @constraint(model, [1:T], y .<= D_max)

    # Add objective
    @objective(model, Max, 2*dot(λ[1,:],y) - sum(y)
                            - delta * sum(sum(ν[1,t,i]+ν[2,t,i] for i=1:p) for t=1:T-1)
                            - sum(dot(epsilon .+ β0, μ[1,t,:]) for t = 1:T)
                            - sum(dot(epsilon .- β0, μ[2,t,:]) for t = 1:T)) #
    optimize!(model)
    return objective_value(model), getvalue.(y), getvalue.(λ), getvalue.(ν), getvalue.(μ)
end

function master_problem(X0, Xt, y0, D_min, D_max, threshold = 0.1, epsilon = 0.1, delta = 0.1, reg = 1, ρ = 1, max_cuts = 10, verbose=0)
    n, p = size(X0)
    T, p = size(Xt)
    #Z = get_Z(X0)
    L_BD = -10000
    U_BD = 10000
    cuts = 0

    # Create model
    model = Model(with_optimizer(Gurobi.Optimizer, GRB_ENV))#Model(with_optimizer(Gurobi.Optimizer))
    set_optimizer_attribute(model, "OutputFlag", 0)

    # Add variables
    @variable(model, α)
    @variable(model, β[j=1:p])

    #Warm start for β

    β_val0 = l2_regression(X0, y0, ρ)#Random.rand(p)#
    #β_val0 = Random.rand(p)


    #Initialization
    _, y_val0, λ_val0, ν_val0, μ_val0 = R(Xt, D_min, D_max, β_val0, epsilon, delta)

    #First constraint
    @constraint(model, α >= 2*dot(λ_val0[1,:],y_val0) - sum(y_val0)
                            - delta * sum(sum(ν_val0[1,t,i]+ν_val0[2,t,i] for i=1:p) for t=1:T-1)
                            - sum(dot(epsilon .+ β, μ_val0[1,t,:]) for t = 1:T)
                            - sum(dot(epsilon .- β, μ_val0[2,t,:]) for t = 1:T))

    # Add objective
    @objective(model, Min, 1/n*sum((y0[i]-sum(X0[i,j]*β[j] for j=1:p))^2 for i=1:n) + reg*α + ρ*sum(β[j]^2 for j=1:p))

    while cuts < max_cuts && U_BD - L_BD > threshold
        if verbose
            println("Lower: ", L_BD, " Upper: ", U_BD)
        end
        cuts += 1

        #Solve current Master Problem
        β_val, α_val, L_BD = solve_model_benders(model)
        U_OA, y_val, λ_val, ν_val, μ_val = R(Xt, D_min, D_max, β_val, epsilon, delta)

        U_BD = 1/n*sum((y0[i]-sum(X0[i,j]*β_val[j] for j=1:p))^2 for i=1:n) + reg*U_OA + ρ*sum(β_val[j]^2 for j=1:p)

        if U_BD - L_BD > threshold
            @constraint(model, α >= 2*dot(λ_val[1,:],y_val) - sum(y_val)
                            - delta * sum(sum(ν_val[1,t,i]+ν_val[2,t,i] for i=1:p) for t=1:T-1)
                            - sum(dot(epsilon .+ β, μ_val[1,t,:]) for t = 1:T)
                            - sum(dot(epsilon .- β, μ_val[2,t,:]) for t = 1:T))
            if verbose
                println("Cut added")
                println("y_val: ", y_val)
            end
        end
    end
    optimize!(model)
    β_val, α_val, L_BD = solve_model_benders(model)
    U_OA, y_val, λ_val, ν_val, μ_val = R(Xt, D_min, D_max, β_val, epsilon, delta)
    if verbose
        println("Final model Obj value: ", objective_value(model))
        println("Lower: ", L_BD, " Upper: ", U_BD)
        println("Final y: ", y_val)
    end
    return objective_value(model), getvalue.(β), getvalue.(α), y_val#, getvalue.(ν), getvalue.(μ)
end

master_problem (generic function with 8 methods)

In [212]:
function R2(y_true, y_test)
    SSR = sum(abs2.(y_true.-y_test))
    SST = sum(abs2.(y_true.-mean(y_true)))
    return 1 - SSR/SST
end

function R2_err(err, y_true)
    SSR = sum(abs2.(err))
    SST = sum(abs2.(y_true.-mean(y_true)))
    return 1 - SSR/SST
end


function compute_CVaR(errs, α_risk)
#     '''
#     Compute the Conditional Value at Risk
#     :param X: Either the data matrix X or the errors
#     :param y: the target values
#     :param alpha: the risk value
#     :param beta:
#     :param b0:
#     :param errs: whether you want to input the data matrix X or the errors
#     :return: the model and the CVaR
#     '''
    n = size(errs)[1]
    # Create model
    model = Model(with_optimizer(Gurobi.Optimizer, GRB_ENV))
    set_optimizer_attribute(model, "OutputFlag", 0)

    # Add variables
    @variable(model, τ)
    @variable(model, z[1:n] >= 0)

    # Add objective
    @objective(model, Min, sum(z)/(α_risk * n) + τ)

    @constraint(model, [1:n], errs .- τ .<= z)

    optimize!(model)

    return objective_value(model)
end


function eval_method(X, y, y_true, split_, past, num_past, val, uncertainty, ϵ_inf, δ_inf, last_yT,
        ϵ_l2, δ_l2, ρ, reg, max_cuts, verbose,
        fix_β0, more_data_for_β0, benders, ridge)

    threshold_benders = 0.01
    n, p = size(X)
    split_index = floor(Int,n*split_)
    #TODO change spli_index with max(split inex, 1)
    X0, y0, Xt, yt, yt_true, D_min, D_max = prepare_data_from_y(X, y, split_index-num_past*past+1, num_past*past, val, uncertainty, last_yT)

    β_list0 = zeros(val, p)
    β_listt = zeros(val, p)
    β_listl2 = zeros(val, p)
    β_l2_init = l2_regression(X0,y0,ρ);
    for s=1:val

        if more_data_for_β0
            X0, y0, Xt, yt, yt_true, D_min, D_max = prepare_data_from_y(X, y, split_index-num_past*past+1, s+(num_past-1)*past, past-1, uncertainty, last_yT)
        else
            X0, y0, Xt, yt, yt_true, D_min, D_max = prepare_data_from_y(X, y, s+split_index-num_past*past+1, (num_past-1)*past, past-1, uncertainty, last_yT)
        end


        if benders
            ##TODO handle fix_beta0
            obj, β0_val, α, y_val = master_problem(X0, Xt, y0, D_min, D_max, threshold_benders, ϵ_inf, δ_inf, reg, ρ, max_cuts, verbose)
            _, βt_val = S_primal(Xt, y_val, β0_val, ϵ_inf, δ_inf);
        else
            if ridge
                obj, βt_val, β0_val = master_primal_l2_ridge(X0, Xt, y0, D_min, D_max, ϵ_inf, δ_inf, reg, ρ, ϵ_l2, δ_l2, fix_β0, β_l2_init)
            else
                obj, βt_val, β0_val = master_primal_l2(X0, Xt, y0, D_min, D_max, ϵ_inf, δ_inf, reg, ρ, ϵ_l2, δ_l2, fix_β0, β_l2_init)
            end
        end

        β_listt[s,:] = βt_val[past-1,:]
        β_list0[s,:] = β0_val
        β_l2 = l2_regression(X0,y0,ρ);
        β_listl2[s,:] = β_l2

    end

    X0, y0, Xt, yt, _, D_min, D_max = prepare_data_from_y(X, y, 1, split_index, val, uncertainty, last_yT)
    _, _, _, _, yt_true, _, _ = prepare_data_from_y(X, y_true, 1, split_index, val, uncertainty, last_yT)

    err_0 = [abs(yt_true[s]-dot(Xt[s,:],β_list0[s,:])) for s=1:val]
    err_t = [abs(yt_true[s]-dot(Xt[s,:],β_listt[s,:])) for s=1:val]
    err_baseline = [abs(yt_true[s]-dot(Xt[s,:],β_l2_init)) for s=1:val]
    err_l2 = [abs(yt_true[s]-dot(Xt[s,:],β_listl2[s,:])) for s=1:val]

    println("\n### β0 Baseline ###")
    println("MAE Baseline: ", mean(err_baseline))
    println("CVAR 0.05 :", compute_CVaR(err_baseline, 0.05))
    println("CVAR 0.15 :", compute_CVaR(err_baseline, 0.15))
    println("R2 : ", R2_err(err_baseline, yt_true))

    println("\n### β0 Baseline Retrained ###")
    println("MAE Baseline: ", mean(err_l2))
    println("CVAR 0.05 :", compute_CVaR(err_l2, 0.05))
    println("CVAR 0.15 :", compute_CVaR(err_l2, 0.15))
    println("R2 : ", R2_err(err_l2, yt_true))

    println("\n### β0 Adaptive ###")
    println("MAE 0: ", mean(err_0))
    println("CVAR 0.05 :", compute_CVaR(err_0, 0.05))
    println("CVAR 0.15 :", compute_CVaR(err_0, 0.15))
    println("R2 : ", R2_err(err_0, yt_true))

    println("\n### βt Adaptive ###")
    println("MAE t: ", mean(err_t))
    println("CVAR 0.05 :", compute_CVaR(err_t, 0.05))
    println("CVAR 0.15 :", compute_CVaR(err_t, 0.15))
    println("R2 : ", R2_err(err_t, yt_true))
end


function prepare_data_from_y(X, y, n0, n, m, uncertainty, last_yT = false)

    X0 = Matrix(X[n0:n0+n,:])
    X0[:,1] = ones(n+1)
    y0 = y[n0:n0+n,:][:]

    yt_true = y[n0+n+1:n0+n+m,:][:]
    Xt = Matrix(X[n0+n+1:n0+n+m,:])
    Xt[:,1] = ones(m)
    yt = yt_true
    if last_yT
        yt_true[m] = mean(Xt[m])
    end

    D_min = yt .- uncertainty.*abs.(yt)
    D_max = yt .+ uncertainty.*abs.(yt)

    return X0, y0, Xt, yt, yt_true, D_min, D_max
end


function l2_regression(X, y, rho; solver_output=0)
    n,p = size(X)

    model = Model(with_optimizer(Gurobi.Optimizer, GRB_ENV))
    set_optimizer_attribute(model, "OutputFlag", solver_output)
    set_optimizer_attribute(model, "NonConvex", 2)

    @variable(model,beta[j=1:p])
    @variable(model, sse>=0)
    #@variable(model, reg>=0)
    @constraint(model, sum((y[i]-sum(X[i,j]*beta[j] for j=1:p))^2 for i=1:n) <= sse)
    #@constraint(model, sum(beta[j]^2 for j=1:p)<=reg)
    @objective(model,Min, 1/n*sse + rho*sum(beta[j]^2 for j=1:p))

    optimize!(model)
    #println("Obj ", objective_value(model))
    return value.(beta)
end

l2_regression (generic function with 1 method)

In [98]:




function master_primal_l2(X0, Xt, y0, Dmin, Dmax, epsilon, delta, reg, ρ, ϵ_l2, δ_l2, β0_fix = false, β0_val = 0)

    M = 1000
    n0, p = size(X0)
    n, _ = size(Xt)

    # Create model
    model = Model(with_optimizer(Gurobi.Optimizer, GRB_ENV))
    set_optimizer_attribute(model, "OutputFlag", 0)

    # Add variables
    @variable(model, β[i=1:n,j=1:p])
    @variable(model, b[i=1:n]>=0)
    @variable(model, β0[j=1:p])
    @variable(model, z[i=1:n], Bin)
    @variable(model, y[i=1:n])
    # Add objective
    @objective(model, Min, 1/n0*sum((y0[i]-sum(X0[i,j]*β0[j] for j=1:p))^2 for i=1:n0)
        + reg*sum(b[i] for i=1:n) + ρ*sum(β0[j]^2 for j=1:p))

    #@constraint(model,[i=1:n], y .- dot(X,β) .<= b)
    #@constraint(model,[i=1:n],-y .+ dot(X,β) .<= b)

    if β0_fix
        @constraint(model, β0 .== β0_val)
    end
    @constraint(model, res_plus_min[i=1:n],  - y[i] + dot(Xt[i,:],β[i,:]) <= b[i])
    @constraint(model, res_minus_min[i=1:n],  y[i] - dot(Xt[i,:],β[i,:]) <= b[i])

    @constraint(model,  y .== Dmax.*z + Dmin.*(1 .-z))
    #@constraint(model,  y .== Dmax.*(1 .-z) + Dmin.*z)

    #@constraint(model, bigM[i=1:n], (dot(Xt[i,:],β[i,:])-Dmin[i])^2 + M*z[i] >= (dot(Xt[i,:],β[i,:])-Dmax[i])^2)
    @constraint(model, bigM[i=1:n], (dot(Xt[i,:],β[i,:]) - Dmin[i])^2 - (dot(Xt[i,:],β[i,:])-Dmax[i])^2 + M*z[i] >= 0)


    #@constraint(model, bigM2[i=1:n],  (dot(Xt[i,:],β[i,:])-Dmax[i])^2 + M*(1-z[i]) >= (dot(Xt[i,:],β[i,:])-Dmin[i])^2)

    @constraint(model, diff_plus[i=2:n],   β[i,:] .- β[i-1,:] .<= delta)
    @constraint(model, diff_minus[i=2:n], - β[i,:] .+ β[i-1,:] .<= delta)

    @constraint(model, diff_0_plus[i=1:n],   β[i,:] .- β0 .<= epsilon)
    @constraint(model, diff_0_minus[i=1:n], - β[i,:] .+ β0 .<= epsilon)

    @constraint(model, sq_0[i=1:n], sum((β[i,:] .- β0).^2) .<= ϵ_l2)
    @constraint(model, sq_t[i=2:n], sum((β[i,:] .- β[i-1,:]).^2) .<= δ_l2)

    #@constraint(model, abs_0[i=1:n], sum(abs.(β[i,:] .- β0)) .<= ϵ_l1)
    #@constraint(model, abs_t[i=2:n], sum(abs.(β[i,:] .- β[i-1,:])) .<= δ_l1)

    optimize!(model);
    #println("SUM ", sum(getvalue.(b)))
    #println("Y ", getvalue.(y))
    return objective_value(model), getvalue.(β), getvalue.(β0)
end




function master_primal_l2_ridge(X0, Xt, y0, Dmin, Dmax, epsilon, delta, reg, ρ, ϵ_l2, δ_l2, β0_fix = false, β0_val = 0)

    M = 1000
    n0, p = size(X0)
    n, _ = size(Xt)

    # Create model
    model = Model(with_optimizer(Gurobi.Optimizer, GRB_ENV))
    set_optimizer_attribute(model, "OutputFlag", 0)

    # Add variables
    @variable(model, β[i=1:n,j=1:p])
    @variable(model, β0[j=1:p])
    @variable(model, z[i=1:n], Bin)
    @variable(model, y[i=1:n])
    # Add objective
    @objective(model, Min, 1/n0*sum((y0[i]-sum(X0[i,j]*β0[j] for j=1:p))^2 for i=1:n0)
        + reg*sum((y[i] - dot(Xt[i,:],β[i,:]))^2 for i=1:n) + ρ*sum(β0[j]^2 for j=1:p))

    #@constraint(model,[i=1:n], y .- dot(X,β) .<= b)
    #@constraint(model,[i=1:n],-y .+ dot(X,β) .<= b)

    if β0_fix
        @constraint(model, β0 .== β0_val)
    end

    @constraint(model,  y .== Dmax.*z + Dmin.*(1 .-z))
    #@constraint(model,  y .== Dmax.*(1 .-z) + Dmin.*z)

    #@constraint(model, bigM[i=1:n], (dot(Xt[i,:],β[i,:])-Dmin[i])^2 + M*z[i] >= (dot(Xt[i,:],β[i,:])-Dmax[i])^2)
    @constraint(model, bigM[i=1:n], (dot(Xt[i,:],β[i,:]) - Dmin[i])^2 - (dot(Xt[i,:],β[i,:])-Dmax[i])^2 + M*z[i] >= 0)


    #@constraint(model, bigM2[i=1:n],  (dot(Xt[i,:],β[i,:])-Dmax[i])^2 + M*(1-z[i]) >= (dot(Xt[i,:],β[i,:])-Dmin[i])^2)

    @constraint(model, diff_plus[i=2:n],   β[i,:] .- β[i-1,:] .<= delta)
    @constraint(model, diff_minus[i=2:n], - β[i,:] .+ β[i-1,:] .<= delta)

    @constraint(model, diff_0_plus[i=1:n],   β[i,:] .- β0 .<= epsilon)
    @constraint(model, diff_0_minus[i=1:n], - β[i,:] .+ β0 .<= epsilon)

    @constraint(model, sq_0[i=1:n], sum((β[i,:] .- β0).^2) .<= ϵ_l2)
    @constraint(model, sq_t[i=2:n], sum((β[i,:] .- β[i-1,:]).^2) .<= δ_l2)

    #@constraint(model, abs_0[i=1:n], sum(abs.(β[i,:] .- β0)) .<= ϵ_l1)
    #@constraint(model, abs_t[i=2:n], sum(abs.(β[i,:] .- β[i-1,:])) .<= δ_l1)

    optimize!(model);
    #println("SUM ", sum(getvalue.(b)))
    #println("Y ", getvalue.(y))
    return objective_value(model), getvalue.(β), getvalue.(β0)
end

master_primal_l2_ridge (generic function with 3 methods)

# Linear rule

In [368]:
function adaptive_ridge_regression_standard(X, y, ρ_β0, ρ_V0, T)
    
#     Adaptive ridge: does not run fast
    

    # Create model
    model = Model(with_optimizer(Gurobi.Optimizer, GRB_ENV))
    set_optimizer_attribute(model, "OutputFlag", 0)
    X, Z, y = get_X_Z_y(X, y, T)

    N, P = size(X)
    # Add variables

    @variable(model, β0[j=1:P])
    @variable(model, V0[j=1:P, k=1:T*P+T])
    @variable(model, t>=0)

    # Add objective
    @objective(model, Min, t + ρ_β0 * sum(β0[j]^2 for j=1:P)
                            + ρ_V0 * sum(V0[j,k]^2 for j=1:P for k=1:T*P+T)
    )

    @constraint(model, t >= 1 / N * sum((y[i] - sum(X[i, j] * (β0[j]
                 + sum(V0[j, l] * Z[i, l] for l=1:(T * P + T))
                 )
                for j=1:P))^2
        for i=1:N))

    optimize!(model);
    return objective_value(model), getvalue.(β0), getvalue.(V0)
end

adaptive_ridge_regression_standard (generic function with 1 method)

In [385]:
function get_X_Z_y(X, y, T)
    """
    Input: training data X and corresponding labels y ; how many time-steps from the past to be used
    Output: the past features X with past targets y as a Z training data (no present features)
    """
    n, p = size(X)
    #T past time steps * p features + T targets
    Z = ones(n-T, T*p+T)
    for i=T+1:n
        for t=1:T
            Z[i-T,1+p*(t-1):p*t] = X[i-t,:]
        end
        Z[i-T, (p*T+1):end] = y[i-T:i-1]
    end
    return X[T+1:end,:], Z, y[T+1:end]
end

get_X_Z_y (generic function with 1 method)

In [370]:
e, Z, f = get_X_Z_y(X[1:5,1:2], y[1:5], 3)

([1.3188635357368816 1.0301917094157778; 1.3699081609434685 1.1463534063406784], [0.6884037525746151 0.5927909913588593 … -0.2486410898872586 0.43491462796847274; 1.3188635357368816 1.0301917094157778 … 0.43491462796847274 0.7909332310183328], [0.7909332310183328, 0.7909332310183328])

In [386]:
_, β0, V0 = adaptive_ridge_regression_standard(X[1:100,1:3], y[1:100], 0.1, 0.1, 3)

(0.02445443166082395, [0.18720284895700318, 0.13539976712246812, 0.10930884939275816], [0.018415765261267765 0.006113858247772819 … -0.02126077974487629 0.005548432581699949; 0.04180042388132868 0.025110266568299063 … -0.008627657122849172 0.026815264905921357; 0.04353282905846895 0.02966493372503107 … 0.015643241897700687 0.03394630338098259])

In [373]:
V0

3×12 Matrix{Float64}:
 0.0184158  0.00611386  -0.014725   …  0.0141299  -0.0212608   0.00554843
 0.0418004  0.0251103    0.0143688     0.0276159  -0.00862766  0.0268153
 0.0435328  0.0296649    0.0211816     0.0305751   0.0156432   0.0339463

In [440]:
function evaluate_adaptive(X, y, y_true, β0, V0, T)
    X, Z, y = get_X_Z_y(X, y, T)
    
    N, P = size(X)

    pred = [sum(X[i, j] * (β0[j] + sum(V0[j, :] .* Z[i, :])) for j=1:P) for i=1:N]
    if size(y_true) == size(pred)
        err = [abs(y_true[i]-pred[i]) for i=1:N]
    else
        y_true = y_true[T+1:end]
        err = [abs(y_true[i]-pred[i]) for i=1:N]
    end
    return err
end

evaluate_adaptive (generic function with 2 methods)

In [449]:
function evaluate_adaptive_retrained(X, y, y_true, β0_list, V0_list, T)
    X, Z, y = get_X_Z_y(X, y, T)
    
    N, P = size(X)

    pred = [sum(X[i, j] * (β0_list[i,j] + sum(V0_list[i, j, :] .* Z[i, :])) for j=1:P) for i=1:N]
    if size(y_true) == size(pred)
        err = [abs(y_true[i]-pred[i]) for i=1:N]
    else
        y_true = y_true[T+1:end]
        err = [abs(y_true[i]-pred[i]) for i=1:N]
    end
    return err
end

evaluate_adaptive_retrained (generic function with 1 method)

In [435]:
pred, err = evaluate_adaptive(X[1:100,1:3], y[1:100], y[4:100], β0, V0, 3)

([0.2772791481270517, -0.22139104249186187, 0.25353125721465, 0.13052228762534424, 0.16184232903639695, 0.02133275894336177, -0.23829147212091226, 0.14926772611852196, 0.28826028113700647, 0.7460128800839689  …  0.6974694989745849, 0.4334507651952026, 0.4535167025297599, 0.4228111533412082, -0.9470370564072363, -0.004111494312688896, 0.12274140243903106, 0.08108513878897414, -1.4025705199739362, 0.08251350395763898], [0.7111257347162706, 0.768332857553357, 0.1994216466416816, 0.6812973681052467, 0.08794262534614797, 0.24054297764516547, 0.02174221306394139, 0.4729100683816176, 0.04127284032870793, 0.5794229469798875  …  0.29188476681020137, 0.3992553152476475, 0.02696830662909544, 0.5024620796474668, 0.7050416471102421, 0.23408638321844036, 0.4254973199898675, 0.7075806393798176, 0.9146849288371715, 0.6178682421677187])

In [484]:
function eval_method(X, y, y_true, split_, past, num_past, val, uncertainty, ϵ_inf, δ_inf, last_yT,
        ϵ_l2, δ_l2, ρ, reg, max_cuts, verbose,
        fix_β0, more_data_for_β0, benders, ridge, linear_adapt)

    threshold_benders = 0.01
    n, p = size(X)
    split_index = floor(Int,n*split_)
    #TODO change spli_index with max(split inex, 1)
    X0, y0, Xt, yt, yt_true, D_min, D_max = prepare_data_from_y(X, y, split_index-num_past*past+1, num_past*past, val, uncertainty, last_yT)

    β_list0 = zeros(val, p)
    β_listt = zeros(val, p)
    β_listl2 = zeros(val, p)
    β_l2_init = l2_regression(X0,y0,ρ);
    β0_list_linear_adapt = zeros(val, p)
    V0_list_linear_adapt = zeros(val, p, (past-1)*p+past-1)
    err = ones(val)
    
    ###Linear decision rule
    _, β0_0, V0_0 = adaptive_ridge_regression_standard(X0, y0, ρ, ρ, past-1)
    print(size(V0))
    
    for s=1:val
        #TODO Check how much I use from the past
        if more_data_for_β0
            X0, y0, Xt, yt, yt_true, D_min, D_max = prepare_data_from_y(X, y, split_index-num_past*past+1, s+(num_past-1)*past, past-1, uncertainty, last_yT)
        else
            X0, y0, Xt, yt, yt_true, D_min, D_max = prepare_data_from_y(X, y, s+split_index-num_past*past+1, (num_past-1)*past, past-1, uncertainty, last_yT)
        end

        if benders
            ##TODO handle fix_beta0
            obj, β0_val, α, y_val = master_problem(X0, Xt, y0, D_min, D_max, threshold_benders, ϵ_inf, δ_inf, reg, ρ, max_cuts, verbose)
            _, βt_val = S_primal(Xt, y_val, β0_val, ϵ_inf, δ_inf);
        else
            if ridge
                obj, βt_val, β0_val = master_primal_l2_ridge(X0, Xt, y0, D_min, D_max, ϵ_inf, δ_inf, reg, ρ, ϵ_l2, δ_l2, fix_β0, β_l2_init)
            else
                obj, βt_val, β0_val = master_primal_l2(X0, Xt, y0, D_min, D_max, ϵ_inf, δ_inf, reg, ρ, ϵ_l2, δ_l2, fix_β0, β_l2_init)
            end
        end
        
        if linear_adapt
            #err[s] = evaluate_adaptive(vcat(X0[end-past+2:end,:],Xt), vcat(y0[end-past+2:end,:],yt), yt_true, β0, V0, past)
            _, β0, V0 = adaptive_ridge_regression_standard(vcat(X0,Xt), vcat(y0,yt), ρ, ρ, past-1)
            β0_list_linear_adapt[s,:] = β0
            V0_list_linear_adapt[s,:,:] = V0
            N = size(X0)[1]
        end
        
        β_listt[s,:] = βt_val[past-1,:]
        β_list0[s,:] = β0_val
        β_l2 = l2_regression(vcat(X0,Xt),vcat(y0,yt),ρ);
        β_listl2[s,:] = β_l2

    end

    X0, y0, Xt, yt, _, D_min, D_max = prepare_data_from_y(X, y, 1, split_index, val, uncertainty, last_yT)
    _, _, _, _, yt_true, _, _ = prepare_data_from_y(X, y_true, 1, split_index, val, uncertainty, last_yT)

    N = size(X0)[1]
    err_linear_rule = evaluate_adaptive(vcat(X0[N-past+2:end,:],Xt), vcat(y0[N-past+2:end,:],yt), yt_true, β0_0, V0_0, past-1)
    err_linear_rule_retrained = evaluate_adaptive_retrained(vcat(X0[N-past+2:end,:],Xt), vcat(y0[N-past+2:end,:],yt), 
        yt_true, β0_list_linear_adapt, V0_list_linear_adapt, past-1)
    
    err_0 = [abs(yt_true[s]-dot(Xt[s,:],β_list0[s,:])) for s=1:val]
    err_t = [abs(yt_true[s]-dot(Xt[s,:],β_listt[s,:])) for s=1:val]
    err_baseline = [abs(yt_true[s]-dot(Xt[s,:],β_l2_init)) for s=1:val]
    err_l2 = [abs(yt_true[s]-dot(Xt[s,:],β_listl2[s,:])) for s=1:val]

    println("\n### β0 Baseline ###")
    println("MAE Baseline: ", mean(err_baseline))
    println("CVAR 0.05 :", compute_CVaR(err_baseline, 0.05))
    println("CVAR 0.15 :", compute_CVaR(err_baseline, 0.15))
    println("R2 : ", R2_err(err_baseline, yt_true))

    println("\n### β0 Baseline Retrained ###")
    println("MAE Baseline: ", mean(err_l2))
    println("CVAR 0.05 :", compute_CVaR(err_l2, 0.05))
    println("CVAR 0.15 :", compute_CVaR(err_l2, 0.15))
    println("R2 : ", R2_err(err_l2, yt_true))
    
    println("\n### β0 V0 Linear rule Adaptive ###")
    println("MAE Baseline: ", mean(err_linear_rule))
    println("CVAR 0.05 :", compute_CVaR(err_linear_rule, 0.05))
    println("CVAR 0.15 :", compute_CVaR(err_linear_rule, 0.15))
    println("R2 : ", R2_err(err_linear_rule, yt_true))
    
    println("\n### β0 V0 Linear rule Adaptive Retrained ###")
    println("MAE Baseline: ", mean(err_linear_rule_retrained))
    println("CVAR 0.05 :", compute_CVaR(err_linear_rule_retrained, 0.05))
    println("CVAR 0.15 :", compute_CVaR(err_linear_rule_retrained, 0.15))
    println("R2 : ", R2_err(err_linear_rule_retrained, yt_true))

    println("\n### β0 Adaptive ###")
    println("MAE 0: ", mean(err_0))
    println("CVAR 0.05 :", compute_CVaR(err_0, 0.05))
    println("CVAR 0.15 :", compute_CVaR(err_0, 0.15))
    println("R2 : ", R2_err(err_0, yt_true))

    println("\n### βt Adaptive ###")
    println("MAE t: ", mean(err_t))
    println("CVAR 0.05 :", compute_CVaR(err_t, 0.05))
    println("CVAR 0.15 :", compute_CVaR(err_t, 0.15))
    println("R2 : ", R2_err(err_t, yt_true))
end

eval_method (generic function with 3 methods)

In [485]:
linear_adapt = true
split_ = N0/(N0+Nt)
past = 5
num_past = floor(Int, Nt/past)

last_yT = false
max_cuts = 10
verbose = false

uncertainty = 0.#1
δ_inf = 0.02
ϵ_inf = 0.02

ϵ_l2 = 0.05
δ_l2 = 0.05

reg = 1/(past*num_past)
ρ = 0.1
val = Nt-1; #n-split_index;

fix_β0 = false
more_data_for_β0 = false
benders = false
ridge = true

eval_method(X, y, y_true, split_, past, num_past, val, uncertainty, ϵ_inf, δ_inf, last_yT, 
        ϵ_l2, δ_l2, ρ, reg, max_cuts, verbose, 
        fix_β0, more_data_for_β0, benders, ridge, linear_adapt)

(3, 12)
### β0 Baseline ###
MAE Baseline: 0.36407079601439907
CVAR 0.05 :0.8510795192177238
CVAR 0.15 :0.7466824544592758
R2 : -0.1550215017223442

### β0 Baseline Retrained ###
MAE Baseline: 0.323850200854258
CVAR 0.05 :0.8483059374560837
CVAR 0.15 :0.6673628477345281
R2 : 0.08364335314613436

### β0 V0 Linear rule Adaptive ###
MAE Baseline: 0.4537841583788847
CVAR 0.05 :0.7043645703192598
CVAR 0.15 :0.6707402082410436
R2 : -0.4229831005260869

### β0 V0 Linear rule Adaptive Retrained ###
MAE Baseline: 0.3161653386943477
CVAR 0.05 :0.6351686862864883
CVAR 0.15 :0.626427302829083
R2 : 0.20223588899848288

### β0 Adaptive ###
MAE 0: 0.32637771499749
CVAR 0.05 :0.840372966556379
CVAR 0.15 :0.6761183816692053
R2 : 0.0663055424040575

### βt Adaptive ###
MAE t: 0.3197312140562855
CVAR 0.05 :0.8162101269545656
CVAR 0.15 :0.6228385217610699
R2 : 0.14510811464986773
